# Storytelling

In [1]:
import pandas as pd
import numpy as np



/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_7956/2504811429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Creazione del Task (problema con pazienti da dati storici)

In [2]:
from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master

from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

In [3]:
# help(Task)

In [4]:
# Instanziare un oggetto task

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= 300,
            robustness_risk= 0.2,
            robustness_overtime= 60, # TODO check robustness_overtime unit of measure
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
)

In [5]:
# Importare i dati e istanziare un oggetto patient provider

historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [6]:
historical_data_df.urgency

0       3
1       1
2       0
3       3
4       0
       ..
9155    1
9156    4
9157    1
9158    1
9159    4
Name: urgency, Length: 9160, dtype: int64

In [7]:
# Campionatura di singoli pazienti
patient = patient_provider.provide_patient()
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='C', requested_urgency=3)
print(patient)

# Campionatura di singoli pazienti
patient = patient_provider.provide_patient(requested_equipe='B', requested_urgency=3)
print(patient)

Patient id: 3572 
 equipe: G 
 urgency: 2 
 days waiting: 46
Patient id: 1857 
 equipe: C 
 urgency: 3 
 days waiting: 13
Patient id: 6259 
 equipe: B 
 urgency: 3 
 days waiting: 56


In [8]:
# Inserire un insieme di pazienti in task
task.patients = patient_provider.provide_patient_set(quantity=300)

## Previsione del tempo operatorio

In [9]:
from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


In [10]:
patients, training = patient_provider.provide_sets(quantity=10, quantity_training=100)


In [11]:
patients


In [12]:
patients[0].features

array([1, 0, 0, ..., 0.0, 0.0, 0.0], dtype=object)

In [13]:
# instantiating the model, and training

model = NGBNormal(description="test", patients=training)

In [14]:
patients_prediction = model.predict(patients)

In [15]:
patients_prediction[0].uncertainty_profile.param_loc

47.224162430074216

In [16]:
patients_prediction[0].uncertainty_profile.param_scale

14.089926887728705

In [17]:
patients_prediction[0].uncertainty_profile.sample(10)

array([39.66704288, 56.4530394 , 26.83201364, 42.69088718, 34.05202196,
       66.4851825 , 57.16930776, 28.05175119, 40.15633902, 34.20607303])

## Aggiunta dei pazienti con previsione (nuovo task)

In [18]:
num_of_patients = 50

In [19]:
patients, training = patient_provider.provide_sets(quantity=num_of_patients, quantity_training=1000)


In [20]:
model = NGBLogNormal(description="test", patients=training)

patients = model.predict(patients)

In [21]:

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= num_of_patients,
            robustness_risk= 0.2,
            robustness_overtime= 10,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients


In [22]:
patients = task.patients
patients[1].max_waiting_days

360

In [23]:
patients[1].days_waiting

307

In [24]:
print(patients[0])

Patient id: 7197 
 equipe: D 
 urgency: 0 
 nominal duration: 72 
 days waiting: 6


## Aggiunta del master scheduling

In [25]:
or_master_schedule = pd.read_csv("../data/master_schedule_input.csv", sep = ';')
or_master_schedule

,weekday,equipes,room,duration
0,1,"C, D",or1,300
1,1,"D, C",or1,300
2,1,I,or2,600
3,2,"H, F",or1,300
4,2,"D, C",or1,300
5,2,L,or2,600
6,3,"C, D",or1,300
7,3,"F, H",or1,300
8,3,I,or1,300
9,3,"H, F",or2,600


In [26]:
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [27]:
print(master_schedule)

Master 'my master' - 16 blocks on 5 days.


In [28]:
task.master_schedule = master_schedule

In [29]:
for block in master_schedule.blocks:
    print(block)
    print('----\n')
    
    

Master Block number 0
                
Equipe: ['C', 'D']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 1
                
----

Master Block number 1
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 1
                
Number in day: 2
                
----

Master Block number 2
                
Equipe: ['I']
                
Duration: 600
                
Room: or2
                
Weekday: 1
                
Number in day: 3
                
----

Master Block number 3
                
Equipe: ['H', 'F']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 1
                
----

Master Block number 4
                
Equipe: ['D', 'C']
                
Duration: 300
                
Room: or1
                
Weekday: 2
                
Number in day: 2
                
----



## Soluzione del modello - Implementor Adversary

In [30]:
implementor_adversary = ImplementorAdversary(task = task, implementor=StandardImplementor(), adversary=None, description = 'Test')



In [31]:
implementor_adversary._task



In [32]:
implementor_adversary.create_instance()



In [33]:
solution = implementor_adversary.run(10)

implementor
adversary


In [34]:
implementor_adversary._instance.get(None).get('n_pats')

{None: 50}

In [35]:
solution = implementor_adversary.run_implementor()


In [36]:
for pat in (solution._blocks[0].patients):
    print(pat)

Patient id: 135 
 equipe: C 
 urgency: 3 
 nominal duration: 84 
 days waiting: 88
Patient id: 4233 
 equipe: C 
 urgency: 4 
 nominal duration: 85 
 days waiting: 133
Patient id: 4463 
 equipe: D 
 urgency: 2 
 nominal duration: 52 
 days waiting: 56
Patient id: 7064 
 equipe: D 
 urgency: 3 
 nominal duration: 73 
 days waiting: 123


solution._blocks[4].patients[0].uncertainty_profile.param_s

In [37]:
task.master_schedule.blocks

In [38]:
task.patients

In [39]:
task.patients[0].uncertainty_profile

## Creazione dell'ottimizzatore

In [40]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [41]:
chance_constraints_model = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')



In [42]:
chance_constraints_model = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [43]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor, BSImplementor


In [44]:
bs_model = VanillaImplementor(task = task, 
                              implementor=BSImplementor(task = task), 
                              description = 'Test')






In [45]:
bs_model.create_instance()

In [46]:
solution = bs_model.run_implementor()

In [66]:
for pat in (solution._blocks[2].patients):
    print(pat.uncertainty_profile.nominal_value)
    print(pat.uncertainty_profile.param_s)

281.8918165477524
0.3252834201781905
77.3606320257393
0.4230817105615229
105.34059089536085
0.34880319344452676


In [47]:
chance_constraints_model._task



In [48]:
chance_constraints_model.create_instance()



In [49]:
chance_constraints_model._instance.get(None).get('n_pats')

{None: 50}

In [50]:
solution = chance_constraints_model.run_implementor()


In [51]:
chance_constraints_model._instance.get(None).get('f')

{1: 72.99123060695435,
 2: 95.04288574480726,
 3: 111.35005166844253,
 4: 57.94566218788996,
 5: 71.94658992067338,
 6: 109.63547827722087,
 7: 46.20284275007701,
 8: 84.57415624194532,
 9: 91.44072345935848,
 10: 193.30409752760164,
 11: 85.86124840592773,
 12: 50.45001940886659,
 13: 48.1794593609872,
 14: 53.7330721723585,
 15: 106.70157239770458,
 16: 116.53137584012534,
 17: 108.97248969764664,
 18: 46.981531917908896,
 19: 52.394620282148225,
 20: 46.78742186897293,
 21: 81.79132409579192,
 22: 204.88574033198785,
 23: 54.84126483271296,
 24: 77.10443566368518,
 25: 71.98020454488127,
 26: 106.84704165378372,
 27: 85.36206521417066,
 28: 115.20261033656946,
 29: 54.23100414463426,
 30: 101.42651379274861,
 31: 47.550319596841526,
 32: 50.61509832396679,
 33: 281.89213245344143,
 34: 77.36257931810114,
 35: 105.34156291871611,
 36: 58.862971063025896,
 37: 116.11297532913193,
 38: 77.50763575048576,
 39: 88.78849193631066,
 40: 102.08371603409213,
 41: 103.2098312017267,
 42: 81.3

In [52]:
for pat in (solution._blocks[0].patients):
    print(pat)

Patient id: 842 
 equipe: D 
 urgency: 3 
 nominal duration: 48 
 days waiting: 65
Patient id: 6089 
 equipe: D 
 urgency: 1 
 nominal duration: 53 
 days waiting: 30
Patient id: 5747 
 equipe: D 
 urgency: 4 
 nominal duration: 46 
 days waiting: 186
Patient id: 4570 
 equipe: D 
 urgency: 4 
 nominal duration: 47 
 days waiting: 242
Patient id: 696 
 equipe: D 
 urgency: 4 
 nominal duration: 50 
 days waiting: 24


In [53]:
solution._blocks[0].patients[0].uncertainty_profile

In [54]:
solution.x[1,1]()

AttributeError: 'Schedule' object has no attribute 'x'

In [ ]:
task.master_schedule.blocks

In [ ]:
task.patients

In [ ]:
task.patients[0].uncertainty_profile